In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [2]:
tr_mcc_codes = pd.read_csv('tables/tr_mcc_codes.csv', sep=';')
tr_types = pd.read_csv('tables/tr_types.csv', sep=';')
transactions = pd.read_csv('tables/transactions.csv', sep=',', nrows=1000000)
gender_train = pd.read_csv('tables/gender_train.csv', sep=',')

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [3]:
x = transactions.tr_type.sample(n=1000)
x[x.isin(tr_types[tr_types.tr_description.str.contains('POS|АТМ')].tr_type)].count() / 1000

0.622

## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [4]:
x = transactions.tr_type.value_counts().sort_values(ascending=False)
res = pd.DataFrame({
    'count' : x.iloc[:10].values,
    'tr_type' : x.iloc[:10].index,
    'tr_description' : tr_types.set_index('tr_type').loc[x.iloc[:10].index].reset_index().tr_description
})
res

,count,tr_type,tr_description
0,231117,1010,Покупка. POS ТУ СБ РФ
1,151166,2010,Выдача наличных в АТМ Сбербанк России
2,149006,7070,Перевод на карту (с карты) через Мобильный бан...
3,137658,1110,Покупка. POS ТУ Россия
4,118975,1030,Оплата услуги. Банкоматы СБ РФ
5,49830,2370,Списание с карты на карту по операции <перевод...
6,28972,7010,Взнос наличных через АТМ (в своем тер.банке)
7,26078,7030,Перевод на карту (с карты) через АТМ (в предел...
8,15781,7071,Перевод на карту (с карты) через Мобильный бан...
9,15476,1100,Покупка. ТУ Россия


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [5]:
cust_tr = transactions.groupby('customer_id').amount.agg([('expenses' , lambda x : x[x < 0].sum()) , ('income' , lambda x : x[x > 0].sum())])
rich_cust = cust_tr[cust_tr.income == cust_tr.income.max()]
poor_cust = cust_tr[cust_tr.expenses == cust_tr.expenses.min()]
print(f'Max income diff:\n{abs(rich_cust.income + rich_cust.expenses)}\n\nMax expenses diff:\n{abs(poor_cust.income + poor_cust.expenses)}')

Max income diff:
customer_id
70780820    1837317.98
dtype: float64

Max expenses diff:
customer_id
70780820    1837317.98
dtype: float64


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [6]:
tr_groups = transactions[transactions.tr_type.isin(res.tr_type)].groupby('tr_type').amount
res_t1 = pd.DataFrame({
    'mean' : tr_groups.mean(),
    'median' : tr_groups.median()
})
res_t1

,mean,median
tr_type,,
1010,-19784.748640,-7411.52
1030,-5320.980222,-2245.92
1100,-44061.827262,-10188.26
1110,-32119.330371,-11207.57
2010,-136077.629325,-44918.32
2370,-205418.249032,-44918.32
7010,276391.789596,112295.79
7030,86104.332909,13951.52
7070,65569.831700,11319.42


In [63]:
tr_groups_rich = transactions.groupby('tr_type')
res_t2 = pd.DataFrame({
    'mean' : tr_groups_rich.apply(lambda x: (x[x.customer_id == rich_cust.index[0]]).amount.mean()).fillna(0),
    'median' : tr_groups_rich.apply(lambda x: (x[x.customer_id == rich_cust.index[0]]).amount.median()).fillna(0)
})
res_t2

,mean,median
tr_type,,
1000,0.000000,0.00
1010,0.000000,0.00
1030,0.000000,0.00
1100,0.000000,0.00
1110,0.000000,0.00
...,...,...
7074,22811.593077,9387.93
7075,0.000000,0.00
8100,0.000000,0.00


In [19]:
tr_groups_rich = transactions[transactions.customer_id == rich_cust.index[0]].groupby('tr_type').amount
res_t2 = pd.DataFrame({
    'mean' : tr_groups_rich.mean(),
    'median' : tr_groups_rich.median()
})
res_t2

,mean,median
tr_type,,
2010,-2.941056e+06,-3368873.660
2330,-2.382398e+06,-2245915.770
2370,-2.218505e+06,-2245915.770
6110,1.756293e+04,10028.010
7020,1.465129e+04,10319.985
7021,2.850067e+04,28500.670
7030,1.406196e+04,8848.910
7031,1.921036e+04,12992.620
7034,1.430648e+04,14306.480


In [64]:
tr_groups_poor = transactions.groupby('tr_type')
res_t2 = pd.DataFrame({
    'mean' : tr_groups_poor.apply(lambda x: (x[x.customer_id == poor_cust.index[0]]).amount.mean()).fillna(0),
    'median' : tr_groups_poor.apply(lambda x: (x[x.customer_id == poor_cust.index[0]]).amount.median()).fillna(0)
})
res_t2

,mean,median
tr_type,,
1000,0.000000,0.00
1010,0.000000,0.00
1030,0.000000,0.00
1100,0.000000,0.00
1110,0.000000,0.00
...,...,...
7074,22811.593077,9387.93
7075,0.000000,0.00
8100,0.000000,0.00


In [8]:
tr_groups_poor = transactions[transactions.customer_id == poor_cust.index[0]].groupby('tr_type').amount
res_t2 = pd.DataFrame({
    'mean' : tr_groups_poor.mean(),
    'median' : tr_groups_poor.median()
})
res_t2

,mean,median
tr_type,,
2010,-2.941056e+06,-3368873.660
2330,-2.382398e+06,-2245915.770
2370,-2.218505e+06,-2245915.770
6110,1.756293e+04,10028.010
7020,1.465129e+04,10319.985
7021,2.850067e+04,28500.670
7030,1.406196e+04,8848.910
7031,1.921036e+04,12992.620
7034,1.430648e+04,14306.480


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [9]:
transactions_full = pd.merge(transactions, gender_train, how='left')
transactions_full = pd.merge(transactions_full, tr_mcc_codes, how='inner')
transactions_full = pd.merge(transactions_full, tr_types, how='inner')
transactions_full.shape

(999584, 9)

In [10]:
transactions_full

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,mcc_description,tr_description
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
1,39026145,6 07:08:31,4814,1030,-5614.79,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
2,39026145,8 07:06:10,4814,1030,-1122.96,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
3,39026145,11 08:49:03,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
4,39026145,11 14:12:08,4814,1030,-2245.92,NaN,1.0,"Звонки с использованием телефонов, считывающих...",Оплата услуги. Банкоматы СБ РФ
...,...,...,...,...,...,...,...,...,...
999579,36837532,18 23:38:60,3000,6000,1297824.89,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999580,6808459,206 19:00:10,3000,6000,124693.24,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999581,83759123,90 22:40:57,3000,6000,218976.79,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ
999582,89619970,104 23:39:35,3000,6000,408307.49,NaN,NaN,"Авиалинии, авиакомпании",Возврат покупки. ТУ СБ РФ


## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [11]:
(abs(transactions_full[(transactions_full.gender == 1) & (transactions_full.amount < 0)].amount.mean() - 
    transactions_full[(transactions_full.gender == 0) & (transactions_full.amount < 0)].amount.mean()), 
abs(transactions_full[(transactions_full.gender == 1) & (transactions_full.amount > 0)].amount.mean() - 
    transactions_full[(transactions_full.gender == 0) & (transactions_full.amount > 0)].amount.mean()))

(32718.05492022467, 63366.57104801465)

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [12]:
gen0_mcc = transactions_full[(transactions_full.gender == 0) & (transactions_full.amount < 0)].groupby(['mcc_code', 'mcc_description']).amount.sum()
gen1_mcc = transactions_full[(transactions_full.gender == 1) & (transactions_full.amount < 0)].groupby(['mcc_code' ,'mcc_description']).amount.sum()

res = pd.DataFrame({
    'diff' : abs(gen0_mcc - gen1_mcc).sort_values(ascending=False).iloc[:10]
})
res.reset_index()

,mcc_code,mcc_description,diff
0,6011,Финансовые институты — снятие наличности автом...,3.085862e+09
1,4829,Денежные переводы,2.499153e+09
2,6010,Финансовые институты — снятие наличности вручную,2.208580e+08
3,5541,Станции техобслуживания,1.554326e+08
4,5511,"Легковой и грузовой транспорт — продажа, серви...",7.352877e+07
5,5812,"Места общественного питания, рестораны",7.037856e+07
6,5977,Магазины косметики,4.241762e+07
7,5533,Автозапчасти и аксессуары,4.233808e+07
8,7995,Транзакции по азартным играм,4.079564e+07
9,5661,Обувные магазины,3.815470e+07


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.

In [13]:
transactions_full['tr_hour'] = transactions_full.tr_datetime.str.extract(r'^\d+ \b(\d{2})').astype('int')
(transactions_full[(transactions_full.gender == 1) & (transactions_full.tr_hour < 7) & (transactions_full.amount < 0)].shape[0],
 transactions_full[(transactions_full.gender == 0) & (transactions_full.tr_hour < 7) & (transactions_full.amount < 0)].shape[0])

(46694, 43510)

In [68]:
transactions_full[(transactions_full.tr_hour < 7) & (transactions_full.amount < 0)].groupby('gender').size()

gender
0.0    43510
1.0    46694
dtype: int64